# Capstone Project

## The Battle of Neighborhoods

### Cultural Similarities
 *Christopher Neal*
 
 ***

# Introduction

The final project will be exploring the cultural similarities of New York and Toronto. Both major cities are labeled as one of the most multicultural cities in the world, but what makes them so diverse from other cities might make them similair to each other. I will be looking at the different types of cultural venues to answer this quesiton. Finding out which business are popular among these two diverse cities and possibly what is more popular in one or the other. If your looking to move to a cultural city with specific interests or trying to open a art gallery to express yourself, which city has what your looking for.

# Data

I will be looking at data on New York and Toronto Neighborhoods, boroughs, latitude, logitude, and cultulral venues such as:
resturants, art, music venue, museum, concert hall, opera house, religious buildings.

New York City data containing the neighborhoods and boroughs, latitudes, and longitudes will be obtained from the data source: https://cocl.us/new_york_dataset

Toronto data will be scrapped from canadas postal code wiki:  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Venue data from both cities are created by using the foursquare developer API

# Methodology


# Problem Statement

Import Libaries

In [34]:
import pandas as pd
import numpy as np
import requests
import folium
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

Foursquare ID

In [35]:
CLIENT_ID = 'HOCKKNRUKYFIFJKAS35RM50FICJ13FBOZHJSTGCFXX5NPD4C' #  Foursquare ID
CLIENT_SECRET = 'WPWAZ3ZCW3ENQAYIF0CJGVOMONT1QFNJ3GLZ5MWDL1I50Q1D' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

Functions

In [36]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    return new_york_data

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except KeyError:
            pass
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Reading datasets

In [76]:
#Previously compiled toronto venues
toronto_venue = pd.read_csv(r'C:\Users\TrustyKarma\github\Projects\Coursera_Capstone\assets\toronto_venues.csv')

# New York Datset
ny_data = get_new_york_data()

Lets look at the New York Dataset

In [38]:
ny_data

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
...,...,...,...,...
301,Manhattan,Hudson Yards,40.756658,-74.000111
302,Queens,Hammels,40.587338,-73.805530
303,Queens,Bayswater,40.611322,-73.765968
304,Queens,Queensbridge,40.756091,-73.945631


In [39]:
ny_venues = getNearbyVenues(names=ny_data['Neighborhood'],
                                   latitudes=ny_data['Latitude'],
                                   longitudes=ny_data['Longitude']
                                  )

In [88]:
#saving dataframe to csv
#ny_venues.to_csv (r'C:\Users\TrustyKarma\github\Projects\Coursera_Capstone\assets\ny_venues.csv', index = False, header=True)

The number of venues is quite large lets simplify to culural venues

In [44]:
print(ny_venues['Venue Category'].unique())

['Dessert Shop' 'Pharmacy' 'Ice Cream Shop' 'Donut Shop' 'Gas Station'
 'Sandwich Place' 'Pizza Place' 'Laundromat' 'Discount Store'
 'Post Office' 'Bagel Shop' 'Grocery Store' 'Fast Food Restaurant'
 'Restaurant' 'Baseball Field' 'Fried Chicken Joint' 'Trail' 'Park'
 'Bus Station' 'Supermarket' 'Accessories Store' 'Caribbean Restaurant'
 'Diner' 'Seafood Restaurant' 'Chinese Restaurant' 'Bowling Alley'
 'Deli / Bodega' 'Food & Drink Shop' 'Platform' 'Metro Station'
 'Convenience Store' 'Bus Stop' 'Plaza' 'River' 'Bank' 'Gym' 'Playground'
 'Gourmet Shop' 'Latin American Restaurant' 'Pub' 'Burger Joint'
 'Spanish Restaurant' 'Beer Bar' 'Warehouse Store' 'Coffee Shop'
 'Mexican Restaurant' 'Bar' 'Wings Joint' 'Thrift / Vintage Store'
 'Bakery' 'Candy Store' 'Breakfast Spot' 'Café' 'Rental Car Location'
 'Supplement Shop' 'Mattress Store' 'Pet Store' 'Shipping Store'
 'Sports Bar' 'Print Shop' 'Furniture / Home Store' 'Liquor Store'
 'Cosmetics Shop' 'Vegetarian / Vegan Restaurant' 'Juice

Obvious cultural venues
* Restaurant (contains word)
* Museum (contains word)
* Music (contains word)
* Historic Site
* Theater (contains word)
* Club (contains word)
* Other Nightlife
* Opera House
* Bar (contains word)

Create a funtion to find the obvious cultural venues

In [72]:
def getCulturalVenues (venues):
     
    #Finding Venues
    restaurant = venues[venues['Venue Category'].str.contains("Restaurant")]
    museum = venues[venues['Venue Category'].str.contains("Museum")]
    music = venues[venues['Venue Category'].str.contains("Music")]
    historic = venues[venues['Venue Category'].str.contains("Historic Site")]
    club = venues[venues['Venue Category'].str.contains("Club")]
    nightlife = venues[venues['Venue Category'].str.contains("Other Nightlife")]
    opera = venues[venues['Venue Category'].str.contains("Opera House")]
    bar = venues[venues['Venue Category'].str.contains("Bar")]
    
    #Creating Dataframe
    culture = pd.concat([restaurant,museum,music,historic,club,nightlife,opera,bar])
    
    return culture

In [73]:
ny_culture = getCulturalVenues(ny_venues)

In [74]:
ny_culture['Venue Category'].unique()

array(['Fast Food Restaurant', 'Restaurant', 'Caribbean Restaurant',
       'Seafood Restaurant', 'Chinese Restaurant',
       'Latin American Restaurant', 'Spanish Restaurant',
       'Mexican Restaurant', 'Vegetarian / Vegan Restaurant',
       'Italian Restaurant', 'Indian Restaurant', 'American Restaurant',
       'Sushi Restaurant', 'French Restaurant', 'Tapas Restaurant',
       'African Restaurant', 'Greek Restaurant',
       'Puerto Rican Restaurant', 'Asian Restaurant',
       'Peruvian Restaurant', 'South American Restaurant',
       'Middle Eastern Restaurant', 'Arepa Restaurant',
       'Eastern European Restaurant', 'Japanese Restaurant',
       'Mediterranean Restaurant', 'Thai Restaurant', 'Cuban Restaurant',
       'Comfort Food Restaurant', 'Caucasian Restaurant',
       'New American Restaurant', 'Vietnamese Restaurant',
       'Dim Sum Restaurant', 'Shabu-Shabu Restaurant',
       'Hotpot Restaurant', 'Polish Restaurant', 'Szechuan Restaurant',
       'Korean Restaur

In [81]:
ny_culture.shape

(3204, 7)

Now lets look at the Toronto Dataset

In [82]:
print(toronto_venue['Venue Category'].unique())

['Bakery' 'Coffee Shop' 'Distribution Center' 'Restaurant' 'Spa' 'Park'
 'Pub' 'Historic Site' 'Breakfast Spot' 'Gym / Fitness Center'
 'Farmers Market' 'Chocolate Shop' 'Performing Arts Venue' 'Dessert Shop'
 'Mexican Restaurant' 'Theater' 'French Restaurant' 'Café' 'Yoga Studio'
 'Event Space' 'Shoe Store' 'Ice Cream Shop' 'Art Gallery'
 'Cosmetics Shop' 'Electronics Store' 'Bank' 'Beer Store' 'Hotel'
 'Wine Shop' 'Antique Shop' 'Portuguese Restaurant' 'Italian Restaurant'
 'Beer Bar' 'Sushi Restaurant' 'Persian Restaurant' 'Creperie'
 'Arts & Crafts Store' 'Hobby Shop' 'Diner' 'Fried Chicken Joint'
 'Chinese Restaurant' 'Smoothie Shop' 'Sandwich Place' 'Gym'
 'College Auditorium' 'Bar' 'Music Venue' 'Clothing Store' 'Comic Shop'
 'Pizza Place' 'Plaza' 'Burrito Place' 'Movie Theater' 'Thai Restaurant'
 'Burger Joint' 'Shopping Mall' 'Sporting Goods Shop'
 'New American Restaurant' 'Japanese Restaurant' 'Gastropub'
 'Fast Food Restaurant' 'Tanning Salon' 'Ramen Restaurant' 'Steakhouse

In [83]:
toronto_culture = getCulturalVenues(toronto_venue)

In [86]:
toronto_culture['Venue Category'].unique()

array(['Restaurant', 'Mexican Restaurant', 'French Restaurant',
       'Portuguese Restaurant', 'Italian Restaurant', 'Sushi Restaurant',
       'Persian Restaurant', 'Chinese Restaurant', 'Thai Restaurant',
       'New American Restaurant', 'Japanese Restaurant',
       'Fast Food Restaurant', 'Ramen Restaurant',
       'Middle Eastern Restaurant', 'Modern European Restaurant',
       'Seafood Restaurant', 'Ethiopian Restaurant',
       'Vietnamese Restaurant', 'American Restaurant',
       'Latin American Restaurant', 'Vegetarian / Vegan Restaurant',
       'German Restaurant', 'Comfort Food Restaurant', 'Asian Restaurant',
       'Moroccan Restaurant', 'Belgian Restaurant', 'Greek Restaurant',
       'Indian Restaurant', 'Falafel Restaurant', 'Korean Restaurant',
       'Colombian Restaurant', 'Brazilian Restaurant',
       'Mediterranean Restaurant', 'Gluten-free Restaurant',
       'Cuban Restaurant', 'Malay Restaurant', 'Caribbean Restaurant',
       'Dim Sum Restaurant', 'Cajun 

In [94]:
toronto_culture.shape

(509, 7)

Now lets compare the cities by neighborhood